In [7]:
# Módulos utilizados
import pandas as pd
import numpy as np


from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

In [8]:
# Leitura dos arquivos, preenchendo valores faltantes com -1
train = pd.read_csv('train.csv', index_col='sku').fillna(-1)
test = pd.read_csv('test.csv', index_col='sku').fillna(-1)

In [9]:
# Separação de atributos de entrada (X) e saída (y)
X_train, y_train = train.drop('isBackorder', axis=1), train['isBackorder']

In [10]:
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.20, random_state=seed)

In [11]:
pca = PCA()
rf = RandomForestClassifier(n_estimators=30, random_state=64, n_jobs=-1)

blackbox_model = Pipeline([('pca', pca), ('rf', rf)])
blackbox_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('rf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=30, n_jobs=-1,
                                        oob_score=False, random_state=64,
                                        verbose=0, warm_

In [12]:
from interpret import show
from interpret.perf import ROC

blackbox_perf = ROC(blackbox_model.predict_proba).explain_perf(X_test, y_test, name='Blackbox')
show(blackbox_perf)

<!-- http://127.0.0.1:7101/1809122634056/ -->

In [13]:
from interpret.blackbox import LimeTabular
from interpret import show

#Blackbox explainers need a predict function, and optionally a dataset
lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train, random_state=1)

#Pick the instances to explain, optionally pass in labels if you have them
lime_local = lime.explain_local(X_test[:5], y_test[:5], name='LIME')

show(lime_local)

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning:

Ill-conditioned matrix (rcond=4.45231e-27): result may not be accurate.

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning:

Ill-conditioned matrix (rcond=5.72876e-27): result may not be accurate.

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning:

Ill-conditioned matrix (rcond=5.77946e-27): result may not be accurate.

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning:

Ill-conditioned matrix (rcond=6.53251e-43): result may not be accurate.

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning:

Ill-conditioned matrix (rcond=5.71372e-27): result may not be accurate.



<!-- http://127.0.0.1:7101/1808352640584/ -->

In [15]:
from interpret.blackbox import ShapKernel
import numpy as np
feature_names = list(X_train.columns)

background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=background_val, feature_names=feature_names)
shap_local = shap.explain_local(X_test[:5], y_test[:5], name='SHAP')
show(shap_local)

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\shap\explainers\kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\shap\explainers\kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\shap\explainers\kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

c:\users\klysm_000\appdata\local\programs\python\python37\lib\site-packages\shap\explainers\kernel.py:555: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will 

<!-- http://127.0.0.1:7101/1808692595336/ -->

In [16]:
from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=blackbox_model.predict_proba, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Global Sensitivity")

show(sensitivity_global)

<!-- http://127.0.0.1:7101/1809122635464/ -->

In [17]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global(name='Partial Dependence')

show(pdp_global)

<!-- http://127.0.0.1:7101/1808720100808/ -->

In [18]:
show([blackbox_perf, lime_local, shap_local, sensitivity_global, pdp_global])

<!-- http://127.0.0.1:7101/1808701605384/ -->
 Open in new window

In [19]:
# Predição da probabilidade de falta (y) para novos valores
y_pred = model.predict_proba(test)[:,1]

NameError: name 'model' is not defined

In [ ]:
# Cria e salva arquivo para submissão
test['isBackorder'] = y_pred
pred = test['isBackorder'].reset_index()
pred.to_csv('submissionML2-1.csv',index=False)